# Spotify API Calls

In [2]:
import pandas as pd
import numpy as np
import json
import os
from pandas.io.json import json_normalize
import re

In [3]:
os.sys.path.append(os.path.dirname(os.path.abspath('.')))

In [4]:
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

In [130]:
cl_id = '66dded11300a4c5a9d565000fa13d8c8'
cl_sec = '1b74966ace0a4560bd7164f323a1afa6'
username = 'antonstorozhilov'
redirect_uri = 'http://localhost'
scope = 'user-read-currently-playing user-read-recently-played user-library-read playlist-modify-public playlist-read-private'
token = util.prompt_for_user_token(username, scope, client_id=cl_id, client_secret=cl_sec, redirect_uri=redirect_uri)
                                   #redirect_uri=redirect_uri)
    

In [131]:
spotify = spotipy.Spotify(auth=token)

### Load and clean datasets

In [7]:
with open('fifty_played_export.json', 'r') as f:
    recent_50_played = json_normalize(json.loads(f.read()))[['track.id', 'played_at', 'track.name', 'track.artists', 'track.album.name', 'track.album.id', 'track.album.images']]

In [8]:
recent_50_played['track.artists.names'] = recent_50_played['track.artists'].apply(lambda x: list(map(lambda y: y['name'], x)))

In [9]:
recent_50_played['track.artists.id'] = recent_50_played['track.artists'].apply(lambda x: list(map(lambda y: y['id'], x)))

In [10]:
recent_50_played=recent_50_played.drop(['track.artists'], axis=1)

In [11]:
recent_50_played

,track.id,played_at,track.name,track.album.name,track.album.id,track.album.images,track.artists.names,track.artists.id
0,1r1Y2hq7wMKe0CRYRkdWFw,2020-01-11T18:16:54.026Z,Side Effects,Honest,5rbqupVSYwvEmXBUnFtUf7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Future],[1RyvyyTE3xzB2ZywiAwp0i]
1,4EpDZxSApKaod3PVqT5ben,2020-01-11T18:14:15.560Z,"Blood, Sweat, Tears",Honest,5rbqupVSYwvEmXBUnFtUf7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Future],[1RyvyyTE3xzB2ZywiAwp0i]
2,2XjcBGdV3h5NJ1hq5NFCy0,2020-01-11T18:07:28.658Z,Special,Honest,5rbqupVSYwvEmXBUnFtUf7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","[Future, Young Scooter]","[1RyvyyTE3xzB2ZywiAwp0i, 4cmA2QDPdzh8W2ytpyUGes]"
3,73pbvYLtPlT6frmLHAmPV8,2020-01-11T18:02:29.558Z,Covered N Money,Honest,5rbqupVSYwvEmXBUnFtUf7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Future],[1RyvyyTE3xzB2ZywiAwp0i]
4,4PjP1pyA9pDDByAheDumTH,2020-01-11T17:58:54.403Z,I Be U,Honest,5rbqupVSYwvEmXBUnFtUf7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Future],[1RyvyyTE3xzB2ZywiAwp0i]
5,11eVz92Nm7b2CTgA8OJVwp,2020-01-11T17:55:01.581Z,Never Satisfied,Honest,5rbqupVSYwvEmXBUnFtUf7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","[Future, Drake]","[1RyvyyTE3xzB2ZywiAwp0i, 3TVXtAsR1Inumwj472S9r4]"
6,2T4JE91t7Js1DJ4rSHltcQ,2020-01-11T17:53:06.065Z,I Won,Honest,5rbqupVSYwvEmXBUnFtUf7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","[Future, Kanye West]","[1RyvyyTE3xzB2ZywiAwp0i, 5K4W6rqBFWDnAN6FQUkS6x]"
7,5ik5nMy8vdS2rXe7SAGt2K,2020-01-11T17:49:06.458Z,Honest,Honest,5rbqupVSYwvEmXBUnFtUf7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",[Future],[1RyvyyTE3xzB2ZywiAwp0i]
8,0ZZWXHDXn6zWDsnz6KSfGQ,2020-01-11T17:45:45.919Z,My Momma,Honest,5rbqupVSYwvEmXBUnFtUf7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","[Future, Wiz Khalifa]","[1RyvyyTE3xzB2ZywiAwp0i, 137W8MRPWKqSmrBGDBFSop]"
9,7GEidxhfGNteiW9n0kLyxH,2020-01-11T17:42:05.284Z,Move That Dope,Honest,5rbqupVSYwvEmXBUnFtUf7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","[Future, Pharrell Williams, Pusha T, Casino]","[1RyvyyTE3xzB2ZywiAwp0i, 2RdwBSPQiwcmiDo9kixcl..."


### Process features

In [11]:
song_features = pd.DataFrame()

In [13]:
for i, v in recent_50_played.iterrows():
    audio_feat = spotify.audio_features(v['track.id'])[0]
    audio_feat['track_id'] = v['track.id']
    audio_feat['track_name'] = v['track.name']
    audio_feat['track_artists'] = v['track.artists.names'] 
    
    genres_list = list()
    for i in v['track.artists.id']:
        genres_list.append(spotify.artist(i)['genres'])
    audio_feat['track_genres'] = genres_list
    
    song_features = song_features.append(audio_feat, ignore_index=True)

In [14]:
song_features = song_features.drop(['analysis_url', 'track_href', 'uri', 'id', 'type', 'duration_ms'], axis=1)

In [15]:
song_features= song_features.set_index('track_id')

In [16]:
song_features

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_artists,track_genres,track_name,valence
track_id,,,,,,,,,,,,,,,
1r1Y2hq7wMKe0CRYRkdWFw,0.21300,0.644,0.7480,0.000000,8.0,0.4350,-4.993,1.0,0.0595,130.004,4.0,[Future],"[[atl hip hop, hip hop, pop rap, rap, southern...",Side Effects,0.7590
4EpDZxSApKaod3PVqT5ben,0.61700,0.633,0.8670,0.000000,7.0,0.4080,-4.995,1.0,0.2140,112.985,4.0,[Future],"[[atl hip hop, hip hop, pop rap, rap, southern...","Blood, Sweat, Tears",0.4040
2XjcBGdV3h5NJ1hq5NFCy0,0.02370,0.785,0.5660,0.000000,4.0,0.1190,-4.371,0.0,0.0886,135.121,4.0,"[Future, Young Scooter]","[[atl hip hop, hip hop, pop rap, rap, southern...",Special,0.0995
73pbvYLtPlT6frmLHAmPV8,0.02930,0.790,0.7230,0.000000,2.0,0.1370,-3.785,1.0,0.0455,125.118,4.0,[Future],"[[atl hip hop, hip hop, pop rap, rap, southern...",Covered N Money,0.1910
4PjP1pyA9pDDByAheDumTH,0.17100,0.580,0.5860,0.000000,4.0,0.1370,-7.968,1.0,0.0791,150.015,4.0,[Future],"[[atl hip hop, hip hop, pop rap, rap, southern...",I Be U,0.4680
11eVz92Nm7b2CTgA8OJVwp,0.50100,0.616,0.6230,0.000008,9.0,0.2430,-7.233,1.0,0.3080,143.888,4.0,"[Future, Drake]","[[atl hip hop, hip hop, pop rap, rap, southern...",Never Satisfied,0.4580
2T4JE91t7Js1DJ4rSHltcQ,0.74300,0.606,0.6600,0.000000,9.0,0.3960,-5.788,1.0,0.2580,83.746,4.0,"[Future, Kanye West]","[[atl hip hop, hip hop, pop rap, rap, southern...",I Won,0.3170
5ik5nMy8vdS2rXe7SAGt2K,0.03260,0.719,0.4770,0.000000,6.0,0.2180,-6.439,0.0,0.1780,140.141,4.0,[Future],"[[atl hip hop, hip hop, pop rap, rap, southern...",Honest,0.2970
0ZZWXHDXn6zWDsnz6KSfGQ,0.04780,0.846,0.4910,0.000227,1.0,0.1000,-7.039,0.0,0.2240,127.007,4.0,"[Future, Wiz Khalifa]","[[atl hip hop, hip hop, pop rap, rap, southern...",My Momma,0.1880


# Grouping

Build distance function and threshold s.t. two songs with distance below threshold belong in the same playlist.

Gold records: 
1. *some* of my existing playlists.
2. Build similar song pairs and groups

### Building Gold records

In [17]:
gold_records_df = pd.DataFrame()

In [12]:
user_playlists = spotify.current_user_playlists(offset=0)

In [13]:
user_playlists.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [14]:
user_playlists['total']

94

In [15]:
playlists_ls = list()

In [16]:
for i in user_playlists['items']:
    playlists_ls.append((i['name'],i['id']))

In [17]:
user_playlists = spotify.current_user_playlists(offset=50)

In [18]:
for i in user_playlists['items']:
    playlists_ls.append((i['name'],i['id']))

In [19]:
filter_playlists_ls = ['Lean & "Mumble Rappers"', 'Downtempo, Rhythm & Blues', 
                  'Shady shit', 'Soul', 'Classic Pop Rock', 'Emo Rock', 'Rough Dance Music', 'Old School Pop Dance Music',
                 'Pop Dance Music', 'Electronica Future Garage', 'Electronica Swing', 'evening', '$.¢¢ ( New Trap )',
                 'Mellow Rap', 'True Trap', 'Fired Up Rap', "Frank's Tracks", 'Old School Pop', 'Caramel', 'Sadbois',
                 'Coding Memes for Classically Savvy Teens', 'Slow Flow', 'Hard Flow']

In [127]:
gold_playlists = list(filter(lambda x: x[0] in filter_playlists_ls, playlists_ls))

In [128]:
gold_playlists

[('Lean & "Mumble Rappers"', '5jNo6WAVnaJOCqbNVu7ALj'),
 ('Downtempo, Rhythm & Blues', '7KXYh1wp6pGJiFUBOOG6IS'),
 ('Shady shit', '5q5oLyj0sUTgfBEeFDdyPH'),
 ('Soul', '1qBueVCuDMnJrrX5GzBprP'),
 ('Classic Pop Rock', '4TBT2oW9RoXH6puCruNYI3'),
 ('Emo Rock', '5rNSNSvbF4WMnpTHvd6fyq'),
 ('Rough Dance Music', '2IqSpWcaiKeuMunIucWwwm'),
 ('Old School Pop Dance Music', '2bmIi01l27EncrUpFGayOF'),
 ('Pop Dance Music', '32OOHk4JZT94kTVPIGCvLO'),
 ('Electronica Future Garage', '27O8ZsECcfj9zYI5AgGR3b'),
 ('Electronica Swing', '5QuuZf3yTnFc1zzReKKpxf'),
 ('evening', '38CaGv6SaYzm4BtmBRHAcg'),
 ('$.¢¢ ( New Trap )', '5F8CAP4L3Qpbmb7XF4Nmzl'),
 ('Mellow Rap', '03Ul56GbmdXNqu9U00Zuge'),
 ('True Trap', '6cY1JABN8ZAF4ZdcmjQ1cY'),
 ('Fired Up Rap', '6g9YAru8WLtTrpnHQ3UK5K'),
 ("Frank's Tracks", '51qtvgmejUvumw5hcsYghY'),
 ('Old School Pop', '4j2OSozqzHWY1wm23K96SU'),
 ('Caramel', '1uHMKaESrpnHMXANlu86gg'),
 ('Sadbois', '4g74pTpCm2wRaxtHvuNZ43'),
 ('Coding Memes for Classically Savvy Teens', '2psiPhL3Yv

In [26]:
# Takes a ['track']
# returns a list of: id, name, artists, genres
def extract_track_info(track_ret):
    
    artists_name_set = set()
    artists_id_set = set()
    
    # Extracting all the artists
    for a in track_ret['artists']:
        artists_name_set.add(a['name'])
        artists_id_set.add(a['id'])
    
    # Extracting genres across artists
    genres_set = set()
    for aid in artists_id_set:
        genres = spotify.artist(aid)['genres']
        for g in genres:
            genres_set.add(g)
    
    
    return [track_ret['id'], track_ret['name'], artists_name_set, genres_set]

In [103]:
def pair_songs_from_playlist(user_id, playlist_id):
    # Get tracks from playlis
    tracks_read_n = 0

    tracks_list = list()
    while(True):
        tracks_ret = spotify.user_playlist_tracks(user_id, playlist_id, offset=tracks_read_n)
        tracks_total = tracks_ret['total']
        tracks_read_n += 50
        if tracks_read_n > tracks_total:
            break

        # Get tracks from playlist
        for t in tracks_ret['items']:
            track_info = extract_track_info(t['track'])
            # Populate song with attributes     
            audio_feat = spotify.audio_features(track_info[0])[0]        
            tracks_list.append( track_info + list(audio_feat.values()))
            
    
    # Pairing tracks
    paired_tracks_list = list()
    for track_1 in tracks_list:
        for track_2 in tracks_list:
            paired_tracks_list.append(track_1 + track_2)
            
            
            
    # Creating new DF
    paired_songs_df = pd.DataFrame.from_records(paired_tracks_list, columns=['track_id', 'name', 'artists', 'genres', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature',
                                                   'track_id_2', 'name_2', 'artists_2', 'genres_2', 'danceability_2', 'energy_2', 'key_2', 'loudness_2', 'mode_2', 'speechiness_2', 'acousticness_2', 'instrumentalness_2', 'liveness_2', 'valence_2', 'tempo_2', 'type_2', 'id_2', 'uri_2', 'track_href_2', 'analysis_url_2', 'duration_ms_2', 'time_signature_2'])
    
    # clean up dataframe
    paired_songs_df = paired_songs_df.drop(['id', 'id_2'], axis=1)
    paired_songs_df = paired_songs_df[paired_songs_df['track_id'] != paired_songs_df['track_id_2']].reset_index(drop=True)
    
    paired_songs_df['related'] = True
    return paired_songs_df

In [104]:
paired_test = pair_songs_from_playlist('antonstorozhilov', '5jNo6WAVnaJOCqbNVu7ALj')

In [125]:
list(gold_playlists)

[]

In [132]:
gold_records_df = pair_songs_from_playlist('antonstorozhilov',gold_playlists[0][1])
for pname, pid in gold_playlists[1:]:
    gold_records_df = gold_records_df.append(pair_songs_from_playlist('antonstorozhilov',pid))

In [137]:
gold_records_df.to_csv("gold_records_df.csv",index=False)

In [43]:
# Get tracks from playlist


tracks_read_n = 0

tracks_list = list()
while(True):
    tracks_ret = spotify.user_playlist_tracks('antonstorozhilov', '5jNo6WAVnaJOCqbNVu7ALj', offset=tracks_read_n)
    tracks_total = tracks_ret['total']
    tracks_read_n += 50
    if tracks_read_n > tracks_total:
        break
   
    # Get tracks from playlist
    for t in tracks_ret['items']:
        track_info = extract_track_info(t['track'])
        # Populate song with attributes     
        audio_feat = spotify.audio_features(track_info[0])[0]        
        tracks_list.append( track_info + list(audio_feat.values()))
        
    
    
    

### Building distance function

'Key' distance research: https://link.springer.com/article/10.1007/s13735-013-0036-6#Sec10

In [ ]:
def song_distance_fn(x_1, x_2, w):
    artist_d = (1-m/(a+b))**2
    genre_d = (1-m/(a+b))**2
    #key_d = 
    time_signature_d = | x_1 - x_2 |
    acousticness_d = | x_1 - x_2 |
    danceability_d = | x_1 - x_2 |
    energy_d = | x_1 - x_2 |
    instrumentalness_d = | x_1 - x_2 |
    liveness_d = | x_1 - x_2 |
    loudness_d = | x_1 - x_2 |
    speechiness_d = | x_1 - x_2 |
    valence_d = | x_1 - x_2 |
    tempo_d = | x_1 - x_2 |
    
    return []

In [ ]:
def song_distance_fn_weight_wrap(x_1, x_2, w)
    return w.T * song_distance_fn(song_distance_fn(x_1, x_2))